In [ ]:
import pandas as pd
df=pd.read_csv("EQUITY_L.csv")
df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,6-Oct-08,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,BE,3-May-95,10,1,INE253B01015,10
2,360ONE,360 ONE WAM LIMITED,EQ,19-Sep-19,1,1,INE466L01038,1
3,3IINFOLTD,3i Infotech Limited,EQ,22-Oct-21,10,1,INE748C01038,10
4,3MINDIA,3M India Limited,EQ,13-Aug-04,10,1,INE470A01017,10


In [ ]:
stock_names = df['NAME OF COMPANY'].tolist()

In [ ]:
#stock_names

In [ ]:
!pip install newsapi-python

In [ ]:
def preprocess_text(text, max_length=512):
    tokens = tokenizer.encode_plus(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')
    return tokens


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load FinBERT model and tokenizer
model_name = 'yiyanghkust/finbert-tone'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")





# Process the text using spaCy
#doc = nlp(result)

# Extract and print named entities
#stock_in_news=set()
#for ent in doc.ents:
 # if ent.label_=="ORG" :
      #print(f"Entity: {ent.text}, Label: {ent.label_}")
  #    stock_in_news.add(ent.text)


In [ ]:
from newsapi import NewsApiClient

# Replace 'YOUR_API_KEY' with your actual News API key
api_key = 'aec19345dff54880a2d38c98254c0905'
newsapi = NewsApiClient(api_key=api_key)


def get_finance_news(keyword, language='en', page_size=100):
  result=""
  news_dict={}
  #predict=[]

  try:
        # Search for news articles based on the provided keyword
    articles = newsapi.get_everything(q=keyword, language=language, page_size=page_size)

        # Display the articles
    for idx, article in enumerate(articles['articles'], start=1):
            #print(f"Article {idx}")
            #print(f"Title: {article['title']}")
            #print(f"Description: {article['description']}")
            #print(f"URL: {article['url']}")
            #print("\n")
            #article['description']=clean_text(article['description'])

            doc = nlp(article['description'])
            for ent in doc.ents:
              if ent.label_=="ORG" :
                for j in stock_names:
                  if ent.text.lower() in j.lower():
                    news_dict[j]=article['description']
                    input_data = preprocess_text(article['description'])
                    break















  except Exception as e:
        print(f"An error occurred: {e}")


  return news_dict



# Example: Get finance news related to the keyword 'stocks'
result=get_finance_news('India Stock Market')


In [ ]:
for i in result.keys():
    for j in stock_names:
        if i.lower() in j.lower().split():
            print(j)


In [ ]:
import re

def clean_text(text):
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    #text = text.lower()

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
len(result)

42

In [ ]:
predict=[]
for input_text in result.values():
    # Preprocess the input text
    input_data = preprocess_text(input_text)

    # Perform inference on the input text
    model.eval()
    with torch.no_grad():
        outputs = model(**input_data)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    # Add the prediction to the list
    predict.append(prediction)

# Display or use the list of predictions as needed
print("List of Predictions:", predict) #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

List of Predictions: [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 2]


In [ ]:
result

{'5Paisa Capital Limited': "Inflation slows but remains above the Fed's target; The resilient U.S. consumer; Stock markets roar back; AI market optimism; Mortgage rates falling; and more",
 'Adani Energy Solutions Limited': 'The Supreme Court said the onus on investigating market-related allegations into the Adani group rested on the government and securities regulator SEBI.',
 'BSE Limited': 'The National Stock Exchange of India, which is larger than the BSE by transaction value, has now taken Hong Kong’s place as the seventh-largest bourse.',
 'Kotak Mahindra Bank Limited': 'Kotak Mahindra group founder has a financial sector model for India: 9% annual growth, $30 trillion GDP by 2047',
 'The Federal Bank  Limited': "Inflation slows but remains above the Fed's target; The resilient U.S. consumer; Stock markets roar back; AI market optimism; Mortgage rates falling; and more",
 'Adani Ports and Special Economic Zone Limited': 'The biggest Nifty losers included Adani Ports, Adani Enterp

In [ ]:
data = {
    'Comapany': list(result.keys()),
    'News': list(result.values()),
    'Score':predict
}
df_final = pd.DataFrame(data)
df_final['Score'] = df_final['Score'].replace(2, -1)


In [ ]:
df_sample = df_final.groupby('Score', group_keys=False).apply(lambda x: x.sample(n=1))

In [ ]:
df_sample.head()

,Comapany,News,Score
41,Laxmi Goldorna House Limited,Creator commerce has a lot of potential to gro...,-1
34,Baid Finserv Limited,Australian Premium Solar IPO: The stock opened...,0
9,TATA CONSUMER PRODUCTS LIMITED,"Biggest gainers on the Nifty were Coal India, ...",1


In [ ]:
df_final["Score"]==-1

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41     True
Name: Score, dtype: bool